In [1]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt 


dataset = ['EFIFI','Nair'][1]
basedir = ['../../bf_data/EFIGI_catalog/','../../bf_data/Nair_and_Abraham_2010/'][1]
fitsdir = basedir + ['fits_temp_Jan_19/','fits_temp_Dec_28/', 'fits_temp_Feb_3/', 'final/'][3]

out_dir = basedir+'out_final/'
fns_r = glob(fitsdir+"*/*r.fits")
fns_r.sort()

# Generate mask + Initial StatMorph

In [2]:
from astrobf.utils import gen_mask
from astrobf.morph import measure_morph

In [6]:
ls ../../bf_data/Nair_and_Abraham_2010/final

J000002.10p155254.15/  J101658.40p613252.85/  J134107.77p561326.53/
J000007.81m000226.09/  J101658.70p493738.31/  J134109.98p520608.99/
J000012.78p010712.72/  J101705.11p544914.06/  J134112.03p014647.32/
J000040.04p143307.72/  J101706.83p093622.56/  J134117.29m024303.01/
J000047.89p141639.09/  J101712.71p131131.31/  J134118.24p520609.90/
J000133.95p150453.64/  J101715.99p532743.16/  J134122.56p050224.41/
J000150.32p010155.24/  J101721.32p534745.96/  J134137.53p541105.84/
J000200.82p150132.23/  J101728.77p060043.07/  J134138.28p583039.42/
J000206.34m101306.79/  J101736.81p062337.67/  J134139.22p554014.22/
J000311.26p155756.61/  J101741.46p014926.76/  J134139.31p050158.33/
J000321.63m095859.83/  J101741.74p402038.97/  J134142.24p405228.06/
J000322.22m104614.25/  J101746.01p104355.26/  J134142.72p041626.65/
J000322.48p010328.42/  J101747.16p090812.52/  J134151.21p020433.92/
J000332.12m104440.78/  J101749.84m004945.49/  J134158.53p465227.66/
J000344.32p161112.76/  J101750.05p421234.58/  J1

J084225.76p371315.73/  J115930.27p532056.00/  J161147.09p501255.90/
J084227.27m000842.60/  J115938.37m032239.43/  J161153.50p291037.92/
J084231.37p402831.24/  J115948.64p102159.45/  J161154.01p301958.00/
J084239.85p404908.08/  J115952.16p553205.78/  J161155.25p294946.86/
J084241.54p424847.42/  J115956.25m003253.55/  J161156.60p292716.31/
J084241.75p524446.93/  J115957.72p084709.50/  J161209.56p413929.50/
J084242.27p360455.21/  J115959.10p523756.52/  J161212.76p385423.76/
J084244.43p355443.08/  J120003.87p560643.49/  J161213.52p401547.98/
J084245.42p292359.84/  J120005.94p525125.16/  J161214.70p483138.10/
J084250.92p540148.52/  J120014.29p104148.27/  J161217.90p255930.83/
J084255.26p360307.35/  J120015.93p524005.48/  J161218.15p290445.59/
J084258.56p360120.75/  J120018.00p001741.95/  J161222.61p525828.30/
J084303.91p511932.53/  J120023.91p002926.39/  J161223.82p320415.68/
J084312.77p373217.60/  J120028.05p093656.26/  J161226.70p381434.47/
J084315.55p034126.52/  J120028.87m000724.86/  J1

In [3]:
gen_mask.run_mask(fns_r, out_dir, npix_min=40)

AttributeError: 'HDUList' object has no attribute 'shape'

In [8]:
hdulist = gen_mask.load_fits(fns_r[0])

In [9]:
hdulist

So far, So good.

In [ ]:
# Run StatMorph for the first time
# Just in case, only for this first run, I pickle all 'morph' instances. a few 10s of GB in total.
measure_morph.run_stat_morph_init(fns[:10], out_dir, eps=1e-6)

# Visual inspection - Filter out bad samples
Keep good samples.
Leave good galaxies (under `.../stat_png/good/*.png`) or list bad galaxies (in trash bin).

In [7]:
partially_filtered = "../../bf_data/Nair_and_Abraham_2010/Intermediate_files/final_150000_done.txt"
partial_bad = "../../bf_data/Nair_and_Abraham_2010/Intermediate_files/final_bad_gal_list.txt"

In [35]:
def gids_from_pnglist(plist):
    """
    Parameters
    ----------
        plist : list of png file names
    """
    gids=[]
    for ll in plist:
        gids.append(ll.split('_')[1])
        
    return gids

In [42]:
with open(partial_bad, 'r') as fbad:
    bad = gids_from_pnglist(fbad.readlines())
    
with open(partially_filtered, 'r') as ff:
    temp = gids_from_pnglist(ff.readlines())

good = []

good_gids=[]
for gg in temp:
    if gg not in bad:
        good.append(gg)

In [41]:
len(good)

9537

# Merge files to minimize I/O

In [50]:
from glob import glob
data_dir = '../../bf_data/Nair_and_Abraham_2010/'

fits_fns = glob(data_dir+'fits_final/*/*-r.fits')
weights_fns = glob(data_dir+'fits_final/*/*-r.weight.fits')
mask_fns = glob(data_dir+'final_all_masks/*_mask.pickle')

assert len(mask_fns) == len(fits_fns) == len(weights_fns)

In [65]:
# Make sure three data sets are in the same order
mask_fns.sort()
fits_fns.sort()
weights_fns.sort()

for fn_fits, fn_weight, fn_mask in zip(fits_fns, weights_fns, mask_fns):
    img_name1 = fn_fits.split("/J")[-2] 
    img_name2 = fn_weight.split("/J")[-2] 
    img_name3 = fn_mask.split("/J")[-1].split('_')[0] # mask names are slightly different
    assert img_name1 == img_name2 == img_name3


In [85]:
# merge three dataset into one. 
# make cutout to save storage space
from astropy.io import fits
import pickle
from astrobf.utils import mask_utils

all_good=[]
for fn_fits, fn_weight, fn_mask in zip(fits_fns, weigths_fns, mask_fns):
    img_name = "J"+fn_fits.split("/J")[-2] 
    if img_name in good:
        img = fits.open(fn_fits)[0].data
        weight = fits.open(fn_weight)[0].data
        mask = pickle.load(open(fn_mask, 'rb'))
        slices = mask_utils._get_stamp_range(mask)
        data = np.stack((img[slices], mask[slices], weight[slices]))
        all_good.append({'data':data, 'img_name':img_name, 'slices':slices})

pickle.dump(all_good, open("all_gals.pickle", "wb"))

## Below under construction!!!

### Following part will be updated if I have to go through this step again.

In [ ]:
# Catalog of StatMorph properties of the good sample.
from astrobf.utils.measure_morph import statMorph_fields
fields = statMorph_fields[:6]
ngal = len(good_gids)
result_arr = np.zeros(ngal, 
                      dtype=[('id','<U24'),('ttype',int),('flag',bool),('flag_sersic',bool)]
                           +[(ff,float) for ff in fields])
i=0
for alm in all_morphs:
    mps = pickle.load(open(alm,'rb'))
    for morph in mps:
        if morph._gid in good_gids:
            result_arr[i]['id'] = morph._gid
            for ff in fields:
                result_arr[i][ff] = getattr(morph, ff)
            i+=1
        else:
            pass

# t-types from catalog

Nair catalog has very specific and inconvenient format. You need a dedicated function for this.

In [168]:
cat = utils.misc.load_Nair('../../bf_data/Nair_and_Abraham_2010/catalog/table2.dat')

inds = cat['ID'].searchsorted(result_arr['id'])
print("Is every element matched?: ", np.all(cat[inds]['ID'] == result_arr['id']))
result_arr['ttype'] = cat[inds]['TT']

# Save the complete catalog
pickle.dump(result_arr, open(out_dir+'result_arr.pcikle', 'wb'))

AttributeError: module 'astrobf.utils' has no attribute 'misc'

# Initialize BO loop -  Measure morphology features

In [ ]:
import torch
import math

# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float

### Initial guesses 


# Run Bayesian Optimization loop